In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load tokenizer dan model IndoBERT
tokenizer = BertTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
model = BertForSequenceClassification.from_pretrained("indobenchmark/indobert-base-p1", num_labels=3)  # num_labels=3 untuk sentimen positif, negatif, netral


In [ ]:
# Contoh teks
texts = ["Saya sangat senang dengan produk ini", "Pelayanan yang buruk sekali", "Produk ini biasa saja"]

# Tokenize teks
inputs = tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="pt")


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['cleaned_text'], df['label'], test_size=0.2, random_state=42)


In [ ]:
from torch.utils.data import Dataset, DataLoader

class SentimentDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.texts.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

train_dataset = SentimentDataset(inputs, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)


In [11]:
from transformers import AdamW

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Set model ke mode training
model.train()

# Training loop
epochs = 3
for epoch in range(epochs):
    for batch in train_dataloader:
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(**batch)
        loss = outputs.loss
        
        # Backward pass
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch+1}/{epochs} - Loss: {loss.item()}")


/home/han/anaconda3/lib/python3.11/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


NameError: name 'train_dataloader' is not defined

In [ ]:
# Set model ke evaluation mode
model.eval()

# Lakukan prediksi pada testing set
with torch.no_grad():
    test_inputs = tokenizer(X_test.tolist(), padding=True, truncation=True, max_length=128, return_tensors="pt")
    test_outputs = model(**test_inputs)
    predictions = torch.argmax(test_outputs.logits, dim=1)

# Hitung akurasi
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, predictions.numpy())
print(f'Akurasi: {accuracy * 100:.2f}%')


In [ ]:
# Save model
model.save_pretrained('path_to_save_model')
tokenizer.save_pretrained('path_to_save_tokenizer')
